In [1]:
import psycopg2
from psycopg2 import sql
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT # <-- ADD THIS LINE
import qqespm_sql_imp as qqsql2
import os

In [ ]:
base_db_name = 'london_pois_5500'
base_csv_filename = 'london_pois_5500'

In [ ]:
percs = ['20perc','40perc','60perc','80perc','100perc']
db_names = [f'{base_db_name}_{perc}' for perc in percs]

In [3]:
connector_info = f"""[postgresql]
host=localhost
database=postgres
user=postgres
password="""

if not os.path.isfile('config/general_connector.ini'):
    with open('config/general_connector.ini', 'w') as f:
        f.write(connector_info)

# NOW, UPDATE THE PASSWORD IN THE GENERATED FILE

In [3]:
conn = qqsql2.establish_postgis_connection(config_filename = 'config/general_connector.ini')
cur = conn.cursor()
for db_name in db_names:
    try:
        cur.execute(sql.SQL("DROP DATABASE IF EXISTS {}").format(sql.Identifier(db_name)))
        cur.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier(db_name)))
        conn.commit()
        print(f'Successfully created or reset database {db_name}')
    except Exception as e:
        print(f'ERROR: {e}')
cur.close()
conn.close()

Successfully created or reset database london_pois_5500_20perc
Successfully created or reset database london_pois_5500_40perc
Successfully created or reset database london_pois_5500_60perc
Successfully created or reset database london_pois_5500_80perc
Successfully created or reset database london_pois_5500_100perc


In [4]:
connector_info = """[postgresql]
host=localhost
database={}
user=postgres
password="""

for db_name in db_names:
    with open(f'config/{db_name}.ini', 'w') as f:
        f.write(connector_info.format(db_name))

# NOW, UPDATE THE PASSWORDS IN THE GENERATED FILES

In [ ]:
csv_files_dir = 'data'

In [5]:
create_pois_table_statement = """CREATE TABLE pois(
osm_id bigint,
name varchar(200),
amenity varchar(50),
shop varchar(50),
tourism varchar(50),
landuse varchar(50),
leisure varchar(50),
building varchar(50),
geometry geometry,
centroid geometry
)
"""

for perc in percs:
    db_name = f'{base_db_name}_{perc}'
    conn = qqsql2.establish_postgis_connection(config_filename = f'config/{db_name}.ini')
    cur = conn.cursor()
    try:
        cur.execute(sql.SQL('create extension hstore'))
        print(f'Successfully created extension hstore on database {db_name}')
        cur.execute(sql.SQL('create extension postgis'))
        print(f'Successfully created extension postgis on database {db_name}')
        cur.execute(sql.SQL(create_pois_table_statement))
        print(f'Successfully created table pois on database {db_name}')
        #import_csv_statement = "COPY pois FROM {} DELIMITERS ',' CSV HEADER"
        #print(import_csv_statement.format(sql.Identifier(csv_files_dir).as_string(), sql.Identifier(base_csv_filename).as_string(), sql.Identifier(perc).as_string()))
        cur.execute(sql.SQL("COPY pois FROM '{}' DELIMITERS ',' CSV HEADER".format(f'{csv_files_dir}/{base_csv_filename}_{perc}.csv')))
        print(f'Successfully imported csv data into table pois on database {db_name}')
        cur.execute(sql.SQL('ALTER TABLE pois ADD COLUMN id SERIAL PRIMARY KEY'))
        print(f'Successfully added primary key column id in table pois on database {db_name}')

        cur.execute(sql.SQL("CREATE INDEX {} ON pois USING GIST ( geometry )").format(sql.Identifier(f'spatial_index_london_{perc}_5500geom_pois')))
        print(f'Successfully created GIST spatial index for geometry column in table pois on database {db_name}')
        cur.execute(sql.SQL("CREATE INDEX {} ON pois USING GIST ( centroid )").format(sql.Identifier(f'spatial_index_london_{perc}_5500centroid_pois')))
        print(f'Successfully created GIST spatial index for centroid column in table pois on database {db_name}')

        cur.execute(sql.SQL("CREATE INDEX {} ON pois USING SPGIST ( geometry )").format(sql.Identifier(f'spatial_index_sp_london_{perc}_5500geom_pois')))
        print(f'Successfully created SPGIST spatial index for geometry column in table pois on database {db_name}')
        cur.execute(sql.SQL("CREATE INDEX {} ON pois USING SPGIST ( centroid )").format(sql.Identifier(f'spatial_index_sp_london_{perc}_5500centroid_pois')))
        print(f'Successfully created SPGIST spatial index for centroid column in table pois on database {db_name}')

        cur.execute(sql.SQL('SET statement_timeout TO 360000000 ;'))
        print(f'Successfully set statement_timeout to 360000000 on database {db_name}')
        print('-----------------------------------------------------')
        conn.commit()
    except Exception as e:
        print(f'ERROR: {e}')
    cur.close()
    conn.close()
    

Successfully created extension hstore on database london_pois_5500_20perc
Successfully created extension postgis on database london_pois_5500_20perc
Successfully created table pois on database london_pois_5500_20perc
Successfully imported csv data into table pois on database london_pois_5500_20perc
Successfully added primary key column id in table pois on database london_pois_5500_20perc
Successfully created GIST spatial index for geometry column in table pois on database london_pois_5500_20perc
Successfully created GIST spatial index for centroid column in table pois on database london_pois_5500_20perc
Successfully created SPGIST spatial index for geometry column in table pois on database london_pois_5500_20perc
Successfully created SPGIST spatial index for centroid column in table pois on database london_pois_5500_20perc
Successfully set statement_timeout to 360000000 on database london_pois_5500_20perc
-----------------------------------------------------
Successfully created extens

In [ ]:
# Testing
db_name = f'{base_db_name}_100perc'
conn = qqsql2.establish_postgis_connection(config_filename = f'config/{db_name}.ini')
cur = conn.cursor()
cur.execute('select * from pois limit 1')
results = cur.fetchall()
cur.close()
results

[(3239540779,
  'Banh Mi Bay',
  'restaurant',
  None,
  None,
  None,
  None,
  None,
  '01030000000100000005000000617138F3AB39B8BFBA6B09F9A0C149407138F3AB3940B8BF560E2DB29DC1494082FFAD64C746B8BFBA6B09F9A0C149407138F3AB3940B8BF1DC9E53FA4C14940617138F3AB39B8BFBA6B09F9A0C14940',
  '01010000007138F3AB3940B8BFBA6B09F9A0C14940',
  1)]